In [6]:
from gc import callbacks

import librosa
import numpy as np
from tqdm.auto import tqdm
import os
import pandas as pd
import time
import soundfile as savefile

## tensor
import pyaudio
import wave
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 
from sklearn.preprocessing import LabelEncoder
import keras

max = 0

In [3]:
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "/Users/anjaeu/Code/졸업프로젝트/new_audio/file.wav"

audio = pyaudio.PyAudio()
# start Recording
stream = audio.open(format=pyaudio.paInt16, 
                    channels=CHANNELS, 
                    rate=RATE, 
                    input=True,
                    frames_per_buffer=CHUNK)

print("recording...")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)
print("finished recording")

# stop Recording

stream.stop_stream()
stream.close()
audio.terminate()


waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
waveFile.setnchannels(CHANNELS)
waveFile.setsampwidth(audio.get_sample_size(FORMAT))
waveFile.setframerate(RATE)
waveFile.writeframes(b''.join(frames))
waveFile.close()

recording...
finished recording


In [4]:
def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        global max
        if mfccs.shape[1] > max:
            max = mfccs.shape[1]

    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return mfccsscaled

def make_dataset(folder) :
    dataset = []
    for file in tqdm(os.listdir(folder),colour='green'):
        if 'wav' in file:
            abs_file_path = os.path.join(folder,file)
            data = extract_features(abs_file_path)
            dataset.append([data])
            
    print("Dataset 생성 완료")
    return pd.DataFrame(dataset,columns=['data'])

In [ ]:
num_labels = 12 # yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy']
              , optimizer = 'adam')

In [7]:
MLP_Model = keras.models.load_model('weights.best.basic_mlp.hdf5')

2022-11-07 00:10:08.250294: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
def print_prediction(file_name):
    
    y = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    le = LabelEncoder()
    le.fit(y)
    
    featuresdf = make_dataset(file_name)
    audio_data = np.array(featuresdf.data.tolist())
    y_prob = MLP_Model.predict(audio_data)
    predicted_vector = np.argmax(y_prob, axis=1)
    predicted_class = le.inverse_transform(predicted_vector)
    if predicted_class[0] == 0 :
        class_name = 'Car Horn'
    elif predicted_class[0] == 1 :
        class_name = 'Dog Bark'
    elif predicted_class[0] == 2 :
        class_name = 'Drilling'
    elif predicted_class[0] == 3 :
        class_name = 'Engine Idling'
    elif predicted_class[0] == 4 :
        class_name = 'Jack Hammer'
    elif predicted_class[0] == 5 :
        class_name = 'Siren'
    elif predicted_class[0] == 6 :
        class_name = 'Bus'
    elif predicted_class[0] == 7 :
        class_name = 'Chime'
    elif predicted_class[0] == 8 :
        class_name = 'Cough'
    elif predicted_class[0] == 9 :
        class_name = 'Knock'
    elif predicted_class[0] == 10 :
        class_name = 'Meow'
    elif predicted_class[0] == 11 :
        class_name = 'Telephone'
    else :
        class_name = "Can't recognize"

    print("The predicted class is:", class_name, '\n') # predicted_class[0]

In [11]:
file_name = "/Users/anjaeu/Code/졸업프로젝트/new_audio"
print_prediction(file_name)

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset 생성 완료
The predicted class is: Telephone 

